In [1]:
import tensorflow as tf
import time
import random
import numpy as np
import os
from collections import defaultdict
from models import tf_Deep_CBow

In [2]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

In [3]:
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [4]:
train = list(read_dataset('../data/classes/train.txt'))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset('../data/classes/test.txt'))
nwords = len(w2i)
ntags = len(t2i)

In [5]:
# initialise model
EMB_SIZE = 64
HID_SIZE = 64
N_LAYERS = 2
model = tf_Deep_CBow(nwords, ntags, EMB_SIZE, N_LAYERS, HID_SIZE)
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [6]:
@tf.function(experimental_relax_shapes=True)
def train_step(words, tag):
    with tf.GradientTape() as tape:
        scores = model(words)
        loss = loss_fn(tag, scores)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [7]:
for epoch in range(10):
    random.shuffle(train)
    start_time = time.time()
    train_loss = 0.0
    for words, tag in train:
        words = tf.constant(words, dtype=tf.float32)
        tag = tf.constant([tag], dtype=tf.float32)
        loss = train_step(words, tag)
    train_loss += loss.numpy()
    print("Epoch %r: train loss/sent=%.4f, time=%.2fs" % (
                epoch, train_loss/len(train), time.time()-start_time))
    test_correct = 0
    for words, tag in dev:
        words = tf.constant(words, dtype=tf.float32)
        tag = tf.constant([tag], dtype=tf.float32)
        scores = model(words)[0].numpy()
        predict = np.argmax(scores)
        if predict == tag:
            test_correct += 1
    print("epoch %r: test acc=%.4f" % (epoch, test_correct/len(dev)))

Epoch 0: train loss/sent=0.0001, time=108.69s
epoch 0: test acc=0.3982
Epoch 1: train loss/sent=0.0002, time=103.08s
epoch 1: test acc=0.3611
Epoch 2: train loss/sent=0.0002, time=107.53s
epoch 2: test acc=0.3864
Epoch 3: train loss/sent=0.0000, time=157.55s
epoch 3: test acc=0.4050
Epoch 4: train loss/sent=0.0001, time=134.65s
epoch 4: test acc=0.3756
Epoch 5: train loss/sent=0.0001, time=130.49s
epoch 5: test acc=0.3910
Epoch 6: train loss/sent=0.0002, time=140.05s
epoch 6: test acc=0.3928
Epoch 7: train loss/sent=0.0001, time=129.39s
epoch 7: test acc=0.3751
Epoch 8: train loss/sent=0.0003, time=123.39s
epoch 8: test acc=0.3688
Epoch 9: train loss/sent=0.0000, time=108.87s
epoch 9: test acc=0.3787
